In [0]:
import tensorflow as tf
print(tf.__version__)
#tf.enable_eager_execution() # for tensorflow version <2x


2.2.0


In [0]:

#!pip install -q tensorflow.datasets # if data set not install

In [0]:
import tensorflow_datasets as tfds
imdb, info=tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGQJR0E/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGQJR0E/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGQJR0E/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

In [0]:
train_data, test_data= imdb['train'], imdb['test']
training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())


for s, l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

#training_sentences[0]

In [0]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)




In [0]:
#hyper parameters
vocab_size=10000
embedding_dim=16
max_length=120
truncate_type="post"
oov_tokn="<OOV>"

# import
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenizer instance
tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tokn)

#tokenizer fit on the sequences
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index

# training sequence or padding
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences, maxlen=max_length, truncating=truncate_type)

# training sequence or padding
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences, maxlen=max_length, truncating=truncate_type)

In [0]:
# now comes the time of neural networks
model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(6, activation="relu"),
      tf.keras.layers.Dense(1,activation="sigmoid")

])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs=10
history=model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4921 - accuracy: 0.7465 - val_loss: 0.3882 - val_accuracy: 0.8235
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2344 - accuracy: 0.9108 - val_loss: 0.4117 - val_accuracy: 0.8215
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0807 - accuracy: 0.9802 - val_loss: 0.5252 - val_accuracy: 0.8055
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0186 - accuracy: 0.9974 - val_loss: 0.6086 - val_accuracy: 0.8066
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0043 - accuracy: 0.9998 - val_loss: 0.6810 - val_accuracy: 0.8071
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.7326 - val_accuracy: 0.8089
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 7.9142e-04 - accuracy: 1.0000 - val_loss: 0.7805 - val_accuracy: 0.8089
Ep

In [0]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
# reverse word_index

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [0]:
import io

out_v=io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]


NameError: ignored

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history["val_"+string])
  plt.xlabel("Epoch")
  plt.ylabel(string)
  plt.legend([string, "val_"+string])
  plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history,"loss")